In [1]:
from itertools import chain

import pandas as pd

import atlas.eugene as eug
from atlas import landowners
from atlas import places

In [2]:
import black
import jupyter_black

jupyter_black.load(
    # lab=False,
    line_length=78,
    # verbosity="DEBUG",
    target_version=black.TargetVersion.PY310,
)

# Eugene: Who Owns the Housing?

In [3]:
assessed_gdf = eug.get_assessed_gdf()

In [4]:
private_owners = assessed_gdf[
    ~assessed_gdf["owner_group"].isin(landowners.public.keys())
]
private_taxpayers = assessed_gdf[
    ~assessed_gdf["taxpayer_group"].isin(landowners.public.keys())
]
private_addresses = assessed_gdf[
    (~assessed_gdf["taxpayer_group"].isin(landowners.public.keys()))
    | (~assessed_gdf["owner_group"].isin(landowners.public.keys()))
]

In [5]:
def get_locality_owned(df):

    if "total_floor_base" in df.columns:
        df = df.copy()
        df["sq_ft"] = df["total_floor_base"]

    in_city = df[df.mailing_city_state_zip.str.contains("EUGENE")]

    in_county_regexes = [
        city for city in places.lane_cities if city != "EUGENE"
    ]
    in_county_regex = "|".join(in_county_regexes)
    in_county = df[
        (df.mailing_city_state_zip.str.contains("OREGON"))
        & (
            df.mailing_city_state_zip.str.contains(
                in_county_regex, regex=True
            )
        )
    ]

    in_state = df[
        (df.mailing_city_state_zip.str.contains("OREGON"))
        & (~df.mailing_city_state_zip.str.contains("EUGENE"))
        & (
            ~df.mailing_city_state_zip.str.contains(
                in_county_regex, regex=True
            )
        )
    ]

    out_of_state_regexes = [
        state for state in places.states if state != "OREGON"
    ]
    out_of_state_regex = "|".join(out_of_state_regexes)
    out_of_state = df[
        df.mailing_city_state_zip.str.contains(out_of_state_regex, regex=True)
    ]

    in_country_regex = "|".join(vars.states)
    out_of_country = df[
        ~df.mailing_city_state_zip.str.contains(in_country_regex, regex=True)
    ]

    def summed(my_df):
        count = pd.Series(data={"count": len(my_df)})
        return pd.concat(
            [
                my_df[
                    [
                        "real_market_value",
                        "sq_ft",
                    ]
                ].sum(),
                count,
            ]
        )

    localities = pd.DataFrame(
        data={
            "Eugene": summed(in_city),
            "Other Lane County": summed(in_county),
            "Other Oregon": summed(in_state),
            "Other USA": summed(out_of_state),
            "International": summed(out_of_country),
        }
    ).T

    totals = localities.sum(numeric_only=True)
    localities.loc["Total"] = totals

    columns = {
        "count": {"percent": "% Props", "name": "Property Count"},
        "real_market_value": {
            "percent": "% RMV",
            "name": "Real Market Value (Millions)",
        },
        "sq_ft": {"percent": "% SqFt", "name": "Square Feet"},
    }

    for k, v in columns.items():
        if k.endswith("value"):
            localities[v["name"]] = localities[k].apply(eug.format_millions)
        else:
            localities[v["name"]] = localities[k].map(lambda x: f"{int(x):,}")
        localities[v["percent"]] = localities[k].map(
            lambda x: eug.format_percent(
                x / localities.head(5)[k].sum(), smart=True
            )
        )

    fields = chain(*[[v["name"], v["percent"]] for v in columns.values()])

    return localities[fields]

In [6]:
mfh = eug.get_multi_family_housing_gdf()

In [7]:
in_state = mfh[
    (mfh.mailing_city_state_zip.str.contains("OREGON"))
    & (~mfh.mailing_city_state_zip.str.contains("EUGENE"))
]

In [8]:
addresses = eug.get_addresses()

# 'UNIT', 'STE', 'BLDG', 'APT'... maybe an apartment?
maybe_mfh_maptaxlots = addresses[
    ~addresses.unit_type_code.isna()
].maptaxlot.unique()

# Pull get those maptaxlots that are in the multi-family-housing GDF
mfh_maptaxlots = mfh[
    mfh.maptaxlot.isin(maybe_mfh_maptaxlots)
].maptaxlot.unique()

# Now look for addresses that have those maptaxlots, or that are 'APT'
apartment_units = addresses[
    (addresses.maptaxlot.isin(mfh_maptaxlots))
    | (addresses.unit_type_code == "APT")
]
# len(apartment_units)
# 26445

## Are MFH Landlords Local?

In [9]:
get_locality_owned(mfh)

,Property Count,% Props,Real Market Value (Millions),% RMV,Square Feet,% SqFt
Eugene,"1,403",45%,"$5,709M",25%,"4,374,887",35%
Other Lane County,381,12%,"$2,566M",11%,"1,167,242",9%
Other Oregon,604,19%,"$5,243M",23%,"2,459,112",20%
Other USA,751,24%,"$9,100M",40%,"4,364,675",35%
International,0,0%,$0M,0%,0,0%
Total,"3,139",100%,"$22,618M",100%,"12,365,916",100%


(Estimated at least 26,445 MFH units, but the tax database does not include this.)

In [10]:
sfh = eug.get_single_family_housing_gdf()

## How Much SFH is Rented?

In [11]:
tenancy = sfh[["Tenancy", "1"]].groupby("Tenancy", observed=True).count()
tenancy["Percent"] = tenancy["1"].map(
    lambda x: round(x / tenancy["1"].sum() * 100)
)
totals = tenancy.sum()
tenancy.loc["Total"] = totals
tenancy["1"] = tenancy["1"].map(lambda x: f"{x:,}")
tenancy["Percent"] = tenancy["Percent"].map(lambda x: f"{x}%")

tenancy.rename(columns={"1": "Houses"})

,Houses,Percent
Tenancy,,
Owner Occupied,"46,047",73%
Rented,"16,786",27%
Total,"62,833",100%


In [12]:
def single_median(series):
    return series.mode().loc[0]

In [13]:
rentals = sfh[sfh["Tenancy"] == "Rented"]
aggregator = {
    "Owner": single_median,
    "Property Count": "sum",
    "real_market_value": "sum",
}
sfh_landlords = (
    rentals[["owner", "1", "real_market_value", "mailing_address"]]
    .rename(
        columns={
            "owner": "Owner",
            "1": "Property Count",
            "mailing_address": "Mailing Address",
        }
    )
    .groupby("Mailing Address")
    .agg(aggregator)
    .sort_values(by="Property Count", ascending=False)
)

sfh_landlords["Real Market Value (Millions)"] = sfh_landlords[
    "real_market_value"
].apply(eug.format_millions)

In [14]:
sfh_owners = (
    sfh[["owner", "1", "real_market_value", "mailing_address"]]
    .rename(
        columns={
            "owner": "Owner",
            "1": "Property Count",
            "mailing_address": "Mailing Address",
        }
    )
    .groupby("Mailing Address")
    .agg(aggregator)
    .sort_values(by="Property Count", ascending=False)
)

sfh_owners["Real Market Value (Millions)"] = sfh_owners[
    "real_market_value"
].apply(eug.format_millions)

## Are SFH Landlords local?

In [15]:
get_locality_owned(rentals)

,Property Count,% Props,Real Market Value (Millions),% RMV,Square Feet,% SqFt
Eugene,"10,344",62%,"$3,195M",62%,"17,428,973",62%
Other Lane County,"1,685",10%,$462M,9%,"2,691,855",10%
Other Oregon,"1,939",12%,$585M,11%,"3,213,450",11%
Other USA,"2,783",17%,$921M,18%,"4,899,780",17%
International,35,0.2%,$12M,0.2%,"54,725",0.2%
Total,"16,786",100%,"$5,175M",100%,"28,288,783",100%


## Who are the Largest SFH Landlords?

In [16]:
sfh_landlords.head(7).drop(columns="real_market_value")

,Owner,Property Count,Real Market Value (Millions)
Mailing Address,,,
"PO BOX 8516 COBURG, OREGON 97408",SOMERSET VILLAS SIG LLC,144,$26M
"1247 VILLARD ST EUGENE, OREGON 97403",SYRIOS TRUST,76,$26M
"3611 DOVE LN EUGENE, OREGON 97402",CARL A PETERSEN LLC,47,$11M
"4061 HARMON LN SPRINGFIELD, OREGON 97478",SMITH SHERRY L TE,44,$9M
"PO BOX 3237 EUGENE, OREGON 97403",OREGON STATE OF,38,$15M
"PO BOX 2617 EUGENE, OREGON 97402",FKM ASSOCIATES LLC,38,$11M
"PO BOX 24608 EUGENE, OREGON 97402",HILYARD TERRACE LTD PTRSHP,35,$9M


In [17]:
top_landlords = sfh_landlords.reset_index().head(20)["Owner"]
top_landlords_gdf = rentals[rentals["owner"].isin(top_landlords)].copy()

## Where are the Largest SFH Portfolios?

In [18]:
eug.explore(
    top_landlords_gdf,
    column="Owner Group",
    legend=True,
    cmap="tab20",
    tooltip=[
        "Owner Group",
        "Taxpayer Group",
        "mailing_address",
        "situs_address",
        "year_built",
    ],
)

## What Size are SFH Landlord Portfolios?

In [19]:
labels = [
    "1",
    "2",
    "3",
    "4",
    "5",
    "6-10",
    "11-20",
    "21-50",
    "51-100",
    "101-150",
]
bins = [0, 1, 2, 3, 4, 5, 10, 20, 50, 100, 150]
binned = pd.cut(
    sfh_landlords.rename(columns={"Property Count": "Portfolio Size"})[
        "Portfolio Size"
    ],
    bins,
    labels=labels,
)
counts = binned.value_counts(sort=False)
pd.DataFrame(counts).rename(columns={"count": "# Landlords"}).reset_index()


landlord_total = len(sfh_landlords)
portfolios_df = pd.DataFrame(counts).rename(columns={"count": "# Landlords"})
portfolios_df["% Landlords"] = portfolios_df["# Landlords"] / landlord_total
portfolios_df["% Landlords"] = portfolios_df["% Landlords"].map(
    lambda x: eug.format_percent(x, smart=True)
)
portfolios_df.loc["Total"] = [landlord_total, "100%"]
portfolios_df

,# Landlords,% Landlords
Portfolio Size,,
1,4340,52%
2,2843,34%
3,426,5%
4,361,4%
5,112,1%
6-10,244,3%
11-20,65,0.8%
21-50,20,0.2%
51-100,1,0.01%


In [20]:
sfh_landlords["Count"] = 1
rental_houses_by_ownership = (
    sfh_landlords[["Property Count", "Count", "real_market_value"]]
    .groupby("Property Count")
    .sum()
    .reset_index()
    .sort_values(by="Property Count", ascending=False)
)
rental_houses_by_ownership["Houses"] = (
    rental_houses_by_ownership["Property Count"]
    * rental_houses_by_ownership["Count"]
)

total_rental_houses = rental_houses_by_ownership.Houses.sum()
total_rental_owners = rental_houses_by_ownership.Count.sum()

In [21]:
for i in range(1, len(rental_houses_by_ownership) + 1):
    print(
        f"{round(rental_houses_by_ownership['Count'].head(i).sum() / total_rental_owners * 1000, 2)}: {i}"
    )

0.12: 1
0.24: 2
0.36: 3
0.48: 4
0.71: 5
0.83: 6
1.07: 7
1.19: 8
1.31: 9
1.43: 10
1.55: 11
1.78: 12
2.14: 13
2.5: 14
2.62: 15
3.09: 16
3.45: 17
3.57: 18
3.92: 19
4.75: 20
5.23: 21
6.3: 22
7.01: 23
8.56: 24
10.34: 25
12.96: 26
15.81: 27
20.56: 28
27.1: 29
39.34: 30
52.66: 31
95.57: 32
146.2: 33
484.13: 34
1000.0: 35


In [22]:
tenth_percent_head_map = {
    1: 7,
    10: 25,
    50: 31,
    100: 32,
    150: 33,
    500: 34,
    1000: 35,
}

In [23]:
def percentilize(df, perc_tenth, head, o_column, p_column, total_properties):
    total_rentals = df[p_column].sum()

    total_owners = df[o_column].sum()

    owners = df.head(head)[o_column].sum()
    owners_pct = eug.format_percent(owners / total_owners, smart=True)

    rentals = df.head(head)[p_column].sum()
    rentals_pct = eug.format_percent(rentals / total_rentals, smart=True)
    properties_pct = eug.format_percent(
        rentals / total_properties, smart=True
    )
    return {
        "Count": owners,
        "Percent": owners_pct,
        "Properties": rentals,
        "Rentals Percent": rentals_pct,
        "Properties Percent": properties_pct,
    }

## What is the Ownership Distribution of SFH Rentals?

In [24]:
ownership_all = [
    percentilize(
        rental_houses_by_ownership,
        k,
        v,
        "Count",
        "Houses",
        total_properties=len(sfh),
    )
    for k, v in tenth_percent_head_map.items()
]

pd.DataFrame(ownership_all).rename(
    columns={
        "Count": "# Landlords",
        "Percent": "% Landlords",
        "Rentals Percent": "% Rental SFH",
        "Properties Percent": "% All SFH",
    }
).set_index("# Landlords")

,% Landlords,Properties,% Rental SFH,% All SFH
# Landlords,,,,
9,0.1%,490,3%,0.8%
87,1%,1719,10%,3%
443,5%,4038,24%,6%
804,10%,5482,33%,9%
1230,15%,6760,40%,11%
4073,48%,12446,74%,20%
8413,100%,16786,100%,27%


## Where is the Publicly Owned Housing?

In [25]:
public_landowners = list(chain(*landowners.public.values()))

mfh_public = mfh[
    (mfh["tax_payer"].isin(public_landowners))
    | (mfh["owner"].isin(public_landowners))
][
    [
        "tax_payer",
        "owner",
        "Owner Group",
        "situs_address",
        "description",
        "geometry",
    ]
]

sfh_public = sfh[
    (sfh["tax_payer"].isin(public_landowners))
    | (sfh["owner"].isin(public_landowners))
][
    [
        "tax_payer",
        "owner",
        "Owner Group",
        "situs_address",
        "description",
        "geometry",
    ]
]

public_housing = pd.concat([mfh_public, sfh_public])

eug.explore(
    public_housing,
    tooltip=True,
    legend=True,
    title=f"{len(public_housing)} Buidings",
    column="Owner Group",
    cmap="Dark2",
)